In [ ]:
import numpy as np
import controlgym as gym
from tqdm import tqdm

In [ ]:
samples = 500
n_states = 100
n_steps = 100
env = gym.make('kuramoto_sivashinsky', n_state = n_states, n_steps = n_steps)
data = {
    't': np.linspace(0, env.n_steps * env.sample_time, env.n_steps + 1),
    'x': env.domain_coordinates,
    'param': np.zeros([samples, 1]),
    'output' : np.zeros([samples, env.n_steps + 1, env.n_state]),
}
for sample in tqdm(range(samples), desc='Generating data'):
    viscosity = np.exp(np.random.uniform(np.log(0.7), np.log(2.0)))
    env = gym.make('kuramoto_sivashinsky', n_state = n_states, n_steps = n_steps, viscosity=viscosity)
    zero_controller = gym.controllers.Zero(env)
    zero_controller.run()
    data['param'][sample] = [viscosity]
    data['output'][sample] = env.state_traj.T.reshape(env.n_steps + 1, env.n_state)

np.save("data_KS_param.npy", data)

In [ ]:
n_state = 100
n_steps = 100
n_action = n_state
samples = 500

# --- Setup ---
env = gym.make('kuramoto_sivashinsky', n_state = n_state, n_steps = n_steps, n_action = n_action, control_sup_width=1/(n_action+1))
domain_coordinates = env.domain_coordinates
sample_time = env.sample_time
times = np.linspace(0, env.n_steps * env.sample_time, env.n_steps + 1)
data = {
    't': times,
    'x': domain_coordinates,
    'output': np.zeros([samples, n_steps + 1, n_state]),
    'forcing': np.zeros([samples, n_steps + 1, 2]),
}
env.close()
    

for sample in tqdm(range(samples), desc='Generating data'):
    env = gym.make('kuramoto_sivashinsky', n_state = n_state, n_steps = n_steps, n_action = n_action, control_sup_width=1/(n_action+1))

    action_history = np.zeros((n_steps + 1, n_action))

    state_history = np.zeros((n_steps + 1, n_state))

    K_a = np.zeros((n_steps+1,n_steps+1))
    K_p = np.zeros((n_steps+1,n_steps+1))
    sigma_a = 0.5
    sigma_p = 4/3
    mu_a = 0.0
    mu_p = 0.0
    tau = 1.0
    times = np.linspace(0, n_steps * env.sample_time, n_steps + 1)

    for i in range(n_steps + 1):
        for j in range(n_steps + 1):
            K_a[i,j] = (sigma_a**2)*np.exp(-((times[i]-times[j])**2/tau**2))
            K_p[i,j] = (sigma_p**2)*np.exp(-((times[i]-times[j])**2/tau**2))

    action_parameters = np.zeros((env.n_steps+1, 2))

    action_parameters[:,0]=np.random.multivariate_normal(mean=np.repeat(mu_a, n_steps+1), cov=K_a)
    action_parameters[:,1]=np.random.multivariate_normal(mean=np.repeat(mu_p, n_steps+1), cov=K_p)
    
    # Get the initial state (s_0)
    _, _ = env.reset()
    state_history[0] = env.state
    for step in range(env.n_steps):
        custom_action = action_parameters[step,0]*np.sin(domain_coordinates/0.5-action_parameters[step,1])

        obs, reward, terminated, truncated, info = env.step(custom_action)
        
        state_history[step + 1] = env.state

        if terminated or truncated:
            break

    data['output'][sample] = state_history
    data['forcing'][sample] = action_parameters
    env.close()
np.save("data_KS_forcing.npy", data)
for key in data:
    print(f"{key}: {data[key].shape}")

In [ ]:
samples = 1000
n_states = 100
n_steps = 100
env = gym.make('burgers', n_state = n_states, n_steps = n_steps)
data = {
    't': np.linspace(0, env.n_steps * env.sample_time, env.n_steps + 1),
    'x': env.domain_coordinates,
    'param': np.zeros([samples, 1]),
    'output' : np.zeros([samples, env.n_steps + 1, env.n_state]),
}
for sample in tqdm(range(samples), desc='Generating data'):
    diffusivity_constant = 0.002*(1 + 18*np.random.rand())
    env = gym.make('burgers', n_state = n_states, n_steps = n_steps, diffusivity_constant=diffusivity_constant)
    zero_controller = gym.controllers.Zero(env)
    zero_controller.run()
    data['param'][sample] = [diffusivity_constant]
    data['output'][sample] = env.state_traj.T.reshape(env.n_steps + 1, env.n_state)

np.save("data_burgers.npy", data)